<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/MIDO_MidiFile_Wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bulk file loading using MIDO library

Re-write of MIDO MIDI file work using python objects

# Setup notebook env

In [1]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# test ..
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root 

In [2]:
# install required libs
!pip install mido


     |████████████████████████████████| 61kB 2.0MB/s 


In [0]:

# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# imports
import pandas as pd
import mido 
from mido import MidiFile



# Python class to wrapper mido.MidiFile objects

Need to clean up code, make it re-usable for parsing multiple MIDI file performance files.



- time signature gathered from MIDI message within file, meta 'time_signature'
- typically 96 - 480 ticks per beat, from 'mido.MidiFile.ticks_per_beat' attribute
- tempo in microseconds per beat, gathered from 'set_tempo' MIDI message in file, converted using mido.tempo2bpm/ bpm2tempo, e.g. 434783 is 138 bpm
- conversion from ticks to seconds -> e.g. mido.tick2second(34, 480, 434783)
microseconds per tick = microseconds per quarter note / ticks per quarter note




NOTE: this next cell is the bulk of the work for this notebook, the rest of the cells are mostly testing for it.

In [0]:
class MIDI_File_Wrapper:
  ''' utility wrapper for loading, parsding a mido.MidiFile object'''

  # column headers for internal data frame
  type_col = 'msg_type'
  time_col = 'delta_time'
  cum_ticks_col = 'ticks_since_start'
  raw_col = 'raw_data'
  cum_ms_col = 'seconds_since_start'


  def __init__(self, file_name):
    self.my_file_name = file_name
    self.my_file_midi = None    
    self.my_tempo = None  # stored as mido.Message instance
    self.my_time_sig = None # stored as mido.Message instance
    self.df_midi_data = None


    # load & configure file info...
    self.parse_file()

  # For call to str(). Prints readable form 
  def __str__(self): 
    return str('file: {}'.format(self.my_file_midi))
    
  
  def parse_file(self):
    ''' file must be: MIDI type 0 only;  one and only one tempo and time_sig meta messages in file. '''

    print('__ FILE: {}'.format(self.my_file_name))

    # load file
    midi_file = MidiFile(self.my_file_name)
    self.my_file_midi = midi_file 

    # make sure it's MIDI type 0 (single track) ...
    if midi_file.type != 0:
      raise ValueError('ERROR! Can only process type 0 files, this file is type: {}'.format(midi_file.type))

    print('tracks: {}'.format(midi_file.tracks))

    # another check for single track ...
    if len(midi_file.tracks) != 1:
      raise ValueError('ERROR! Need a single MIDI track, this file has: {}, {}'.format(midi_file.tracks, midi_file))

    # parse messages for time_sig and tempo info ..
    for msg in midi_file:

      if msg.type == 'time_signature':
        print('time sig: {}'.format(msg))

        # make sure no time sig changes
        if self.my_time_sig != None:
          raise ValueError('ERROR! more than one time sig: {}, {}'.format(self.my_time_sig, msg))
      
        self.my_time_sig = msg

      elif msg.type == 'set_tempo':

        print('tempo: {}'.format(msg))

        # make sure no tempo changes
        if self.my_tempo != None:
          raise ValueError('ERROR! more than one tempo: {}, {}'.format(self.my_tempo, msg))
        
        self.my_tempo = msg

    # now check we actually have tempo and time_sig set, or complain...
    if self.my_time_sig is None:
      raise ValueError('ERROR! no time signature found: {}'.format(midi_file))
    if self.my_tempo is None:
      raise ValueError('ERROR! no tempo found: {}'.format(midi_file))

    # load messages into DF
    self.load_df()

    # additional processing, e.g. cum_sum for clicks, milliseconds from start, etc

  def tempo_ms(self):
    """returns tempo in milliseconds"""
    return self.my_tempo.tempo

  def tempo_bpm(self):
    return mido.tempo2bpm(self.tempo_ms())

  def ticks(self):
    ''' Returns number of MIDI ticks configured in this file'''
    return self.my_file_midi.ticks_per_beat

  def length(self):
    ''' returns running time in seconds'''
    return self.my_file_midi.length

  def msg_counts(self):
    ''' handy for debug '''
    return self.df_midi_data['msg_type'].value_counts()


  def calculate_seconds(self, ticks_since_start):
    """takes elapsed ticks since start of files, returns 
        position in file in seconds"""
    # ticks and tempo saved from file loading time..
    return mido.tick2second(ticks_since_start, self.ticks(), self.tempo_ms())


  def __row_to_seconds(self, row):
    return self.calculate_seconds(row[self.cum_ticks_col])


  def load_df(self):
    df_setup = []

    for msg in self.my_file_midi.tracks[0]:
      df_setup.append(
          {
              self.type_col: msg.dict()['type'],
              self.time_col: msg.dict()['time'],
              self.raw_col:  str(msg.dict())
          } 
      )
      
    df_tmp = pd.DataFrame(df_setup)

    # tweak data types, change from 'object' columns to 'string'  ...
    df_tmp[self.type_col] = df_tmp[self.type_col].astype('string')
    df_tmp[self.raw_col] = df_tmp[self.raw_col].astype('string')

    # add cumulative tick count column, used to store a running total
    # giving time a message appears in the performance/ MIDI file.
    df_tmp[self.cum_ticks_col] = df_tmp[self.time_col].cumsum()

    # add cumulative milliseconds from start of file
    # NOTE: this timing needs to be recalculated if the tempo
    #         is ever changed!!!
    df_tmp[self.cum_ms_col] = df_tmp.apply(self.__row_to_seconds, axis=1)
  
    # store final df
    self.df_midi_data = df_tmp

    


## Testing the above class...


In [5]:
# test loading single file..
file_1 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid'
mfw = MIDI_File_Wrapper(file_1)


__ FILE: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
tracks: [<midi track 'MIDI' 1300 messages>]
time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
tempo: <meta message set_tempo tempo=434783 time=0>


In [6]:
# take a look at what we got..
mfw.df_midi_data.info()
mfw.df_midi_data.tail(10)
mfw.df_midi_data
print('file length in seconds: {}'.format(mfw.length()))
print('tempo in ms: {}'.format(mfw.tempo_ms()))
print('tempo in bpm: {}'.format(mfw.tempo_bpm()))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   msg_type             1300 non-null   string 
 1   delta_time           1300 non-null   int64  
 2   raw_data             1300 non-null   string 
 3   ticks_since_start    1300 non-null   int64  
 4   seconds_since_start  1300 non-null   float64
dtypes: float64(1), int64(2), string(2)
memory usage: 50.9 KB


,msg_type,delta_time,raw_data,ticks_since_start,seconds_since_start
1290,note_on,106,"{'type': 'note_on', 'time': 106, 'note': 38, '...",30448,27.579735
1291,note_on,4,"{'type': 'note_on', 'time': 4, 'note': 51, 've...",30452,27.583358
1292,note_off,107,"{'type': 'note_off', 'time': 107, 'note': 38, ...",30559,27.680279
1293,note_off,4,"{'type': 'note_off', 'time': 4, 'note': 51, 'v...",30563,27.683902
1294,control_change,51,"{'type': 'control_change', 'time': 51, 'contro...",30614,27.730097
1295,control_change,44,"{'type': 'control_change', 'time': 44, 'contro...",30658,27.769953
1296,note_on,0,"{'type': 'note_on', 'time': 0, 'note': 44, 've...",30658,27.769953
1297,control_change,36,"{'type': 'control_change', 'time': 36, 'contro...",30694,27.802561
1298,note_off,77,"{'type': 'note_off', 'time': 77, 'note': 44, '...",30771,27.872308
1299,end_of_track,0,"{'type': 'end_of_track', 'time': 0}",30771,27.872308


,msg_type,delta_time,raw_data,ticks_since_start,seconds_since_start
0,track_name,0,"{'type': 'track_name', 'name': 'MIDI', 'time': 0}",0,0.000000
1,instrument_name,0,"{'type': 'instrument_name', 'name': 'Brooklyn'...",0,0.000000
2,time_signature,0,"{'type': 'time_signature', 'numerator': 4, 'de...",0,0.000000
3,key_signature,0,"{'type': 'key_signature', 'key': 'C', 'time': 0}",0,0.000000
4,smpte_offset,0,"{'type': 'smpte_offset', 'frame_rate': 24, 'ho...",0,0.000000
...,...,...,...,...,...
1295,control_change,44,"{'type': 'control_change', 'time': 44, 'contro...",30658,27.769953
1296,note_on,0,"{'type': 'note_on', 'time': 0, 'note': 44, 've...",30658,27.769953
1297,control_change,36,"{'type': 'control_change', 'time': 36, 'contro...",30694,27.802561
1298,note_off,77,"{'type': 'note_off', 'time': 77, 'note': 44, '...",30771,27.872308


file length in seconds: 27.872307693749978
tempo in ms: 434783
tempo in bpm: 137.99987580011177


## Test loading lots of files, creating lots of custom objects..



In [7]:
# testing parsing sub directory..

# reminder: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/

import os

for dirpath, dirs, files in os.walk('/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/'):
  for f in files:
    print('in subdir: {}, first file: {}'.format(dirpath, f)) 
    break

 

in subdir: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/session2, first file: 4_jazz_120_beat_3-4.mid
in subdir: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/session3, first file: 6_dance-disco_120_beat_4-4.mid
in subdir: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session, first file: 1_funk-groove1_138_beat_4-4.mid
in subdir: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/session1, first file: 123_funk_95_fill_4-4.mid


In [8]:
import glob

root_dir = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'
#root_dir = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/session1/'

# recursively find all MIDI files..
my_files = []
for filename in glob.iglob(root_dir + '**/*.mid', recursive=True):
  my_files.append(filename)

# show what I found
my_files
print()
print('total # files: {}'.format(len(my_files)))


['/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/2_funk-groove2_105_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/4_soul-groove4_80_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/6_hiphop-groove6_87_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/8_rock-groove8_65_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/3_soul-groove3_86_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/9_soul-groove9_105_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/5_funk-groove5_84_beat_4-4


total # files: 10


## Test creating lots of custom objects..

On creating a MIDI_File_Wrapper instance, a bunch of testing is performed, so the following is a good test to check a whole bunch of files at once to confirm bulk loading works, and to verify the files themselves meet criteria checked during loading..

In [9]:

my_midi_files = []

for f in my_files:
  next_midi = MIDI_File_Wrapper(f)
  #print(next_midi.df_midi_data.info())
  my_midi_files.append(next_midi)
  
print('')
print('TOTAL NUM OBJECTS CREATED: {}'.format(len(my_midi_files)))

__ FILE: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
tracks: [<midi track 'MIDI' 1300 messages>]
time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
tempo: <meta message set_tempo tempo=434783 time=0>
__ FILE: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/2_funk-groove2_105_beat_4-4.mid
tracks: [<midi track 'MIDI' 829 messages>]
time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
tempo: <meta message set_tempo tempo=571429 time=0>
__ FILE: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid
tracks: [<midi track 'MIDI' 639 messages>]
time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
tempo: <meta message set_tempo tempo=588235 